In [1]:
# Uniprot gene name mapper

In [2]:
# Sometimes people use the not-standart gene names, e.g. ELMSAN1 should be MIDEAS.
# I map everything to the first identifier of uniprot

In [8]:
import re
def remove_non_alphanumeric(input_string):
    return re.sub(r'[^a-zA-Z0-9]', '', input_string)

In [4]:
import pandas as pd
import numpy as np
import torch
import os
current_directory = os.getcwd()
data_path='/'+os.path.join(*current_directory.split('/')[:-2])+'/non_anndata_data'

/tmp/ipykernel_21216/1570694649.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
# Maps for genes with several symbol the alternative symbols to the main gene symbol
df=pd.read_csv(f'{data_path}/uniprotkb_AND_model_organism_9606_2024_02_19.tsv', sep='\t')
gene_names=list(df['Gene Names'])
Dict={}
for g in gene_names:
    if ' ' in str(g):
        target_gene=g.split(' ')[0]
        for gs in g.split(' ')[1:]:
            Dict[gs]=target_gene
np.save(f'{data_path}/Gene_names_dict.npy', Dict)

In [6]:
Gene_names_dict= np.load(f'{data_path}/Gene_names_dict.npy', allow_pickle=True).item()
Gene_names_dict_keys=list(Gene_names_dict.keys())

In [10]:
embeddings=torch.load(f'{data_path}/gene_vector_averages.pt')
embeddings['control']=torch.zeros_like(list(embeddings.values())[0])
embedding_genes=list(embeddings.keys())
for gene in embedding_genes:
    if gene in Gene_names_dict_keys:
        embeddings[Gene_names_dict[gene]]=embeddings[gene]
genes=list(embeddings.keys())
for gene in genes:
    embeddings[remove_non_alphanumeric(gene)]=embeddings[gene]
torch.save(embeddings, f'{data_path}/gene_name_mapped_embedding.pt')

In [11]:
# And the same thing but now with all 0s, this is used for the baseline
embeddings_zeros=embeddings
for g in embeddings.keys():
    embeddings[g]=torch.zeros_like(embeddings[g])
torch.save(embeddings, f'{data_path}/gene_name_mapped_embedding_all_zero.pt')